In [1]:
# The contents of this module is to focus on the overall control flow of entire module. Not really considering splits.
# O/S = If dealer gets blackjack AND player gets blackjack, everyone loses their bets - needs to be factored into outcome. 

import pandas as pd
import random

In [2]:
columns = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
index = [i for i in range(5, 22)]
index2 = ['A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10']
index3 = ['2, 2','3, 3', '4, 4', '5, 5', '6, 6', '7, 7', '8, 8', '9, 9', '10, 10', 'J, J', 'Q, Q', 'K, K', 'A, A']
indices = index + index2 + index3
basic_strategy_table = pd.read_csv('basic-strategy.csv', header=None)
basic_strategy_table.index = indices
basic_strategy_table.columns = columns

basic_strategy_table_no_doubling = pd.read_csv('basic-strategy-no-doubling.csv', header=None)
basic_strategy_table_no_doubling.index = indices
basic_strategy_table_no_doubling.columns = columns

def getcards(decks=1):
    """Brings out a number of sealed decks of cards in their original order to the virutal gaming table.

    Creates a dictionary tracking the name of each card, its quantity, and its value
    
    Keyword argument:
        decks {int} -- the number of 52-card decks that the casino is using (default 1)
    
    Returns:
        dictionary{keys: values} -- dictionary object where keys are the card name (e.g. 'K' for King )
                                    and where the values are a list object containing the card's quantity and its value.
                                    
    The value for the ace contains three elements, as aces can further take on the value of 1 or 11, depending on 
    the context of the hand. No distinction is made between card suits, i.e. spades, hearts, clubs, diamonds.
    """
    
    deck = {}
    total_cards = decks * 52
    card_quantity = int(total_cards / 13)
    
    deck['A'] = [card_quantity, 1, 11]

    for card in range(2, 11):
        deck[str(card)] = [card_quantity, card]
    
    for card in "JQK":
        deck[str(card)] = [card_quantity, 10]
        
    return deck, total_cards

deck, total_cards = getcards(decks=1)

def shuffler(deck, total_cards):
    
    """Shuffles the decks of cards, and places them inside the virtual gaming table card-shoe."""
    
    shuffled_shoe = []
    
    while len(shuffled_shoe) < total_cards:
        draw = random.choice(list(deck))
        if deck[draw][0] == 0:
            pass
        else:
            deck[draw][0] -= 1
            shuffled_shoe.append(draw)
            
    return shuffled_shoe

shuffled_shoe = shuffler(deck, total_cards)

In [3]:
rigged_shoe = []

for card in "2234A9766366922":
    rigged_shoe.append(card)
print(rigged_shoe)

['2', '2', '3', '4', 'A', '9', '7', '6', '6', '3', '6', '6', '9', '2', '2']


In [4]:
def openinghand():
    
    """Deals two cards to the player, and one to the dealer from the shuffled shoe. Outputs a message if there is Blackjack
    
    Cards are taken sequentially from the shuffled card shoe, beginning with the last element of the shuffled shoe list object
    and working in reverse order."""
    
    player = []
    dealer = []

    player.append(rigged_shoe.pop())
    player.append(rigged_shoe.pop())
    
    dealer.append(rigged_shoe.pop())
    
    if ('A' in player) and (('10' in player) or ('J' in player) or ('Q' in player) or ('K' in player)):
        player_blackjack = 1
        print("Blackjack, house pays out at 3:2")
    else:
        player_blackjack = 0
        print("No Blackjack this time")
        
    return player, dealer, player_blackjack

player, dealer, player_blackjack = openinghand()

No Blackjack this time


In [5]:
def decode(action):
    if action == 'Sp':
        message = 'split'
    elif action == 'H':
        message = 'hit'
    elif action == 'D':
        message = 'double down'
    else:
        message = 'stick'
    return message

In [6]:
def basic_player_multiple_hits(player, dealer, player_blackjack):
    
    if len(player) == 2:
        
        if player_blackjack == 1:
            cardvalues = [21]
            decision = 'S'
            print("It's my lucky day")
        else:
            if player[0] == player[1]:
                lookup_pairs = player[0] + ", " + player[1]
                decision = basic_strategy_table.loc[lookup_pairs, dealer[0]]
                print("As I have a pair of {}s, I am going to {}".format(player[0], decode(decision)))
            elif ('A' in player):
                if player[0] == 'A':
                    lookup_soft_total = player[0] + player[1]
                else:
                    lookup_soft_total = player[1] + player[0]
                
                decision = basic_strategy_table.loc[lookup_soft_total, dealer[0]]
                print("As I have a soft total, I am going to {}".format(decode(decision)))
            else:
                cardvalues = []
                for card in player:
                    cardvalues.append(deck[card][1])
                
                lookup_hard_total = sum(cardvalues)
                decision = basic_strategy_table.loc[lookup_hard_total, dealer[0]]
                print("I have a total of {}, I am going to {}".format(lookup_hard_total, decode(decision)))
        
    if len(player) > 2: 
        
        if 'A' in player:  # check if the >3 hand has an ace, i.e. is potentially soft.
            if player[0] != 'A':  # re-order dual ace to the left
                ace_index = player.index('A')  
                player[0], player[ace_index] = player[ace_index], player[0]
            
            cardvalues_no_ace = [deck[card][1] for card in player[1::]]
            
            if sum(cardvalues_no_ace) <= 10: # check soft totals
                lookup_soft_total = player[0] + str(sum(cardvalues_no_ace))

                decision = basic_strategy_table_no_doubling.loc[lookup_soft_total, dealer[0]]
                print("As I have a soft {}, that is {} or {}, I am going to {}.".format(11 + sum(cardvalues_no_ace), 1 + sum(cardvalues_no_ace), 11 + sum(cardvalues_no_ace), decode(decision)))
            else:
                lookup_hard_total = deck['A'][1] + sum(cardvalues_no_ace) # variable naming, hard total as ace can only take value 1 now
                
                decision = basic_strategy_table_no_doubling.loc[lookup_hard_total, dealer[0]]
                print("I have {}, so I am going to {}".format(1 + sum(cardvalues_no_ace), decode(decision)))
        else:
            cardvalues = [deck[card][1] for card in player]
            lookup_hard_total = sum(cardvalues)
            
            decision = basic_strategy_table_no_doubling.loc[lookup_hard_total, dealer[0]]
            print("I have {}, so I am going to {}".format(sum(cardvalues), decode(decision)))
            
    return decision

In [7]:
def player_action(player, decision):
    
    if decision == 'S':
        print(player)
        return player# -> go to dealerturn()

    if decision == 'D':
        drawcard = rigged_shoe.pop()
        player.append(drawcard)
        print(player) #-> go to dealerturn(), and also have to modify payout
        return player
        
    
    if decision == 'H':
        drawcard = rigged_shoe.pop()
        player.append(drawcard)
        print(player) #-> now there are three cards in player hand and one in dealer
        return player
        

    if decision == 'Sp': 
        multiple_hand_player = [player[i:i+1] for i in range(len(player))] #splits the player hand into two cards and places them in two sub-hands
    
        drawcard1 = shuffled_shoe.pop()
        drawcard2 = shuffled_shoe.pop()
    
        print("After, splitting, I receive {} and {} in each of my hands".format(drawcard1, drawcard2))
    
        multiple_hand_player[0].append(drawcard1)
        multiple_hand_player[1].append(drawcard2)
        
        print("My new hands are: " + str(multiple_hand_player))
        
        return multiple_hand_player

In [8]:
def dealerturn(dealer, player_blackjack):
    
    """Evaluates the dealer's score, and keeps drawing cards until dealer has a score of 17 or more."""
    
    if player_blackjack == 1:
        cardvalues = [deck[dealer[0]][1]]
        print("Dealer draws no cards as player has blackjacked")
    else: 
        cardvalues = []
        for card in dealer:
            cardvalues.append(deck[card][1])
        
        while sum(cardvalues) <= 16:
            drawcard = rigged_shoe.pop()
            dealer.append(drawcard)
            cardvalues.append(deck[drawcard][1])
            print("Dealer draws a card as under or equal to 16")
    
        if sum(cardvalues) > 21:
            print("Dealer goes bust")
        else:
            print("Dealer sticks on 17 or over")
    
        print("Dealer Score:" + str(sum(cardvalues)))
        print(dealer)
    
    return dealer, sum(cardvalues)

In [9]:
def handoutcome(playerscore, dealerscore, player_blackjack):
    
    """Compares player's and dealer's scores and encodes the outcome.
    
    Can be extended if analysis required tracking the evolution of this variable."""
    
    if player_blackjack == 1:
        hand_outcome = 3
        print("Player wins the hand with Blackjack")
    else:
        if playerscore > 21:
            if dealerscore > 21:
                hand_outcome = 0
                print("Player busts. Dealer also busts. But the 'edge' means that player loses the hand.")
            else:
                hand_outcome = 0
                print("Player busts. Dealer wins the hand.")
        else:
            if dealerscore > 21:
                hand_outcome = 1
                print("Dealer busts. Player wins the hand")
            else:
                if dealerscore > playerscore:
                    hand_outcome = 0
                    print("Dealer has higher score. Dealer wins the hand")
                elif dealerscore < playerscore:
                    hand_outcome = 1
                    print("Player has higher score. Player wins the hand")
                else:
                    hand_outcome = 2
                    print("Both dealer and player have the same score, the outcome of the hand is a draw")
    
    return hand_outcome

In [10]:
def payout_loss(hand_outcome):
    
    """Uses the outcome to calculate payoffs/losses, and updates player's capital stock accordingly."""
    
    global capital_stock
    
    if hand_outcome == 0:
        new_capital = capital_stock[-1] - bet_per_round
        capital_stock.append(new_capital)
        print("Player loses " + str(bet_per_round) + " pounds.")
    
    elif hand_outcome == 1:
        new_capital = capital_stock[-1] + bet_per_round
        capital_stock.append(new_capital)
        print("House pays out " + str(bet_per_round) + " pound.s")
        
    elif hand_outcome == 2:
        new_capital = capital_stock[-1]
        capital_stock.append(new_capital)
        print("Draw, no change to player's initial capital.")
        
    else:
        new_capital = capital_stock[-1] + (1.5 * bet_per_round)
        capital_stock.append(new_capital)
        print("Player blackjacks, house pays out " + str(1.5 * bet_per_round) + " pounds.")
        
    print("Player now has" + " " + str(capital_stock[-1]) + " " + "pounds remaining.")
        
    return capital_stock

In [11]:
def replenish_shoe(threshold):
    
    """Checks whether or not  we are near the end of a shoe. If so, discards the remaining cards in the current shoe,
    collects them together with remaining cards that were discarded in previous hands, reshuffles all of them togther,
    and finally replenishes the shoe."""
    
    global deck
    global total_cards
    global shuffled_shoe
    
    fraction_cards_remaining = len(shuffled_shoe) / total_cards
    
    if fraction_cards_remaining < threshold:
        shuffled_shoe.clear
        deck, total_cards = getcards(decks=1)
        shuffled_shoe = shuffler(deck, total_cards)
        print("As we are reaching near the end of the shoe, we will reshuffle")
    else:
        print("No need for reshuffling yet, as card threshold not reached")

    return shuffled_shoe

In [12]:
# Busting module
# O/S - simplify the logic, if possible. Make it a function of a generic argument like hand?

def check_if_bust(player):
    if 'A' in player:
        if player[0] != 'A':
            ace_index = player.index('A')
            player[0], player[ace_index] = player[ace_index], player[0]
            
        card_values_no_ace = [deck[card][1] for card in player[1::]]
        
        if sum(card_values_no_ace) <= 10:
            bust_indicator = 0
        elif sum(card_values_no_ace) > 20:
            bust_indicator = 1
        else:
            bust_indicator = 0
    else:
        card_values = [deck[card][1] for card in player]
        
        if sum(card_values) > 21:
            bust_indicator = 1
        else:
            bust_indicator = 0
            
    return bust_indicator

In [13]:
deck, total_cards = getcards(decks=6)
shuffled_shoe = shuffler(deck, total_cards)

In [14]:
rigged_shoe = shuffled_shoe

In [15]:
rigged_shoe = []

for card in "2A4A95AA9A2AAAA92A622":
    rigged_shoe.append(card)
    
print(rigged_shoe)

['2', 'A', '4', 'A', '9', '5', 'A', 'A', '9', 'A', '2', 'A', 'A', 'A', 'A', '9', '2', 'A', '6', '2', '2']


In [16]:
# Write control flow here. This feels like it has shed a lot of light on issues I was grappling with.
# If decision = {S, D}, then the CF takes over.
# Structuring the decision = {H} and no bust case is trickier - some subtleties regarding control flow.

# Thrown an error message - it's because there is an edge case, A10,
# which has not been specified in the basic strategy lookup table.
# The corresponding rigged shoe setup that throws this error is given above.
# Amended by adding an 'A10' entry to basic strategy table. This needs to be explained in the write-up.

# Need to code up the print messages to better understand under what conditions the loop terminates.

player, dealer, player_blackjack = openinghand()
print("Player's hand is {}.".format(player) + " Dealer's hand is {}.".format(dealer))
decision = basic_player_multiple_hits(player, dealer, player_blackjack)
player = player_action(player, decision)

if (decision == 'S') or (decision =='D'):
    print("go to dealer, CF takes over")
else:
    while decision == 'H':
        bust_indicator = check_if_bust(player)
        
        if bust_indicator == 0:
            print("Continue decision-action sequence")
            decision = basic_player_multiple_hits(player, dealer, player_blackjack)
            player = player_action(player, decision)
        else:
            print("Now dealer's turn as I have gone bust, CF takes over")
            break
    else:
        print("Loop terminated by decision = {S, D}")

No Blackjack this time
Player's hand is ['2', '2']. Dealer's hand is ['6'].
As I have a pair of 2s, I am going to split
After, splitting, I receive 3 and 3 in each of my hands
My new hands are: [['2', '3'], ['2', '3']]
Loop terminated by decision = {S, D}


In [26]:
# Have to deal with issue where the given a player hand, and the dealer receive a picture i.e. Dealer = [J/Q/K], decision module
# looks up dealer[0], and for these cards, it won't be present in the lookup table.

# Can either do a lookup of J/Q/K value in the dictionary and lookup points value '10' in BS table
# OR add columns that are exactly same as '10' in BS table to include J/Q/K.

# Going to add to the BS table, and check against other strategy tables. Also applies to pairs - done.

In [49]:
# This is proposed refinement of above that aims to trim down some redundant conditions.
# Last else that breaks out of the while loop is only triggered when decision == 'S' or 'D'.
# Merge the 1st, that checks whether to initiate while loop with else that breaks out of the while loop; so the
# while is skipped over if decision - {S, D}.

# In CF diagram terms, is this merging the flow from dealer turn onwards 

player, dealer, player_blackjack = openinghand()
print("Player's hand is {}.".format(player) + " Dealer's upcard is {}.".format(dealer))
decision = basic_player_multiple_hits(player, dealer, player_blackjack)
player = player_action(player, decision)

while decision == 'H':
    playerbust_indicator = check_if_bust(player)
        
    if playerbust_indicator == 0:
        print("Continue decision-action sequence")
        decision = basic_player_multiple_hits(player, dealer, player_blackjack)
        player = player_action(player, decision)
    else:
        print("Now dealer's turn as I have gone bust, CF takes over")
        break
else:
    print("Decision-action sequence terminated by decision = {S, D}")
    print("Go to dealer, CF takes over")

No Blackjack this time
Player's hand is ['10', '2']. Dealer's upcard is ['2'].
I have a total of 12, I am going to hit
['10', '2', 'Q']
Now dealer's turn as I have gone bust, CF takes over


In [113]:
# Process: After player has been dealt two cards, he will have to use BS table to look up his decision using the
# basic_player_multiple_hits(player, dealer, player_blackjack) decision module; and then take an action using the 
# player_action(player, decision) module. 

# After this has happened, a mechanism is needed to co-ordinate whether the player's turn ends at that point, and the dealer's
# begins; or whether the player takes another decision-action. 

# That is encapsulated in the playerturn(player, decision) function; which uses the output of the 1st decision-action sequence
# to co-ordinate whether the entire CF of the round should proceed to the dealer's turn or not. 

def player_turn(player, decision):

    while decision == 'H':
        bust_indicator = check_if_bust(player)
        
        if bust_indicator == 0:
            print("Continue decision-action sequence")
            decision = basic_player_multiple_hits(player, dealer, player_blackjack)
            print(decision)
            player = player_action(player, decision)
            print(player)
        else:
            print("Now dealer's turn as I have gone bust, CF takes over")
            break
    else:
        print("Decision-action sequence terminated by decision = {S, D}")
        print("Go to dealer, CF takes over")
        
    return player, decision

In [399]:
# CF with subsequent decision-action modules and their co-ordination bundled into a module that controls when they are invoked. 

player, dealer, player_blackjack = openinghand()
print("Player's hand is {}.".format(player) + " Dealer's upcard is {}.".format(dealer))
decision = basic_player_multiple_hits(player, dealer, player_blackjack)
player = player_action(player, decision)
print(player, decision)
player, decision, playerbust_indicator = player_turn(player, decision)

No Blackjack this time
Player's hand is ['A', '2']. Dealer's upcard is ['9'].
As I have a soft total, I am going to hit
['A', '2', 'A']
['A', '2', 'A'] H
Continue decision-action sequence
As I have a soft 14, that is 4 or 14, I am going to hit.
H
['A', '2', 'A', 'A']
['A', '2', 'A', 'A']
Continue decision-action sequence
As I have a soft 15, that is 5 or 15, I am going to hit.
H
['A', '2', 'A', 'A', 'A']
['A', '2', 'A', 'A', 'A']
Continue decision-action sequence
As I have a soft 16, that is 6 or 16, I am going to hit.
H
['A', '2', 'A', 'A', 'A', 'A']
['A', '2', 'A', 'A', 'A', 'A']
Continue decision-action sequence
As I have a soft 17, that is 7 or 17, I am going to hit.
H
['A', '2', 'A', 'A', 'A', 'A', '2']
['A', '2', 'A', 'A', 'A', 'A', '2']
Continue decision-action sequence
As I have a soft 19, that is 9 or 19, I am going to stick.
S
['A', '2', 'A', 'A', 'A', 'A', '2']
['A', '2', 'A', 'A', 'A', 'A', '2']
Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes ov

In [400]:
player_turn(player, decision)

Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over


(['A', '2', 'A', 'A', 'A', 'A', '2'], 'S', 0)

In [401]:
compute_playerscore(player)

Player has a soft total


19

In [138]:
# Amended above to include playerbust indicator in event of decision = {D}; and now
# make a cosmetic distinction between decision-action sequences being
# terminated by decision = {S} and decision = {D}.

# Note this cannot yet handle decision = {Sp}; covered in another area.

def player_turn(player, decision):

    while decision == 'H':
        playerbust_indicator = check_if_bust(player)
        
        if playerbust_indicator == 0:
            print("Continue decision-action sequence")
            decision = basic_player_multiple_hits(player, dealer, player_blackjack)
            print(decision)
            player = player_action(player, decision)
            print(player)
        else:
            print("Now dealer's turn as I have gone bust, CF takes over")
            break
    else:
        if decision == 'S':
            print("Decision-action sequence terminated by decision = 'S'")
            print("Go to dealer, CF takes over")
            playerbust_indicator = 0
        elif decision == 'D':
            print("Decision-action sequence terminated by decision = 'D'")
            print("Go to dealer, CF takes over")
            playerbust_indicator = check_if_bust(player)
            
            if playerbust_indicator:
                print("Player doubled and busted.")
            else:
                print("Player doubled and did not bust.")
        
    return player, decision, playerbust_indicator

In [172]:
# Working compute_playerscore module - taken from another notebook, and used to test CF

def compute_playerscore(player):
    soft_total_indicator = evaluate_hand(player)
    
    if soft_total_indicator:
        if player[0] != 'A':
            ace_index = player.index('A')
            player[0], player[ace_index] = player[ace_index], player[0]
            
        cardvalues_no_ace = [deck[card][1] for card in player[1::]]
        
        if sum(cardvalues_no_ace) >= 11:
            playerscore = deck['A'][1] + sum(cardvalues_no_ace)
        elif sum(cardvalues_no_ace) <= 10:
            playerscore = deck['A'][2] + sum(cardvalues_no_ace)   
    else:
        playerscore = sum([deck[card][1] for card in player])
        
    return playerscore

In [368]:
# Evaluate player hand module called in compute_playerscore module - taken from another notebook and used to test CF

def evaluate_hand(player):
    if 'A' in player:
        soft_total_indicator = 1
        print("Player has a soft total")
    else:
        soft_total_indicator = 0
        print("Player has a hard total")
    return soft_total_indicator

* The next few offshoots to my thinking are here to show your progress, and for debrief write up purposes. Now you look back you may think it is trivial, but there were some thorny game logical and algorithmic issues you were dealing with at the same time.

In [118]:
# CF has taken over, there will be no fedback decision-action sequences, and the remainder of CF is linear.
# So now we can definitively compute the score of player hand, salvaging parts of the failed attempt below :-) - good one.
# You knew it would come in handy
# Also think about whether this can be generalisable to the evaluation of the dealer's score.

# As there was difficulty in computing playerscore, I decided to do the simpler case of the dealer score, and then the playerscore

def compute_playerscore(player):
    if len(player) == 2:
        if 'A' in player:
            if player[0] != 'A':
                ace_index = player.index('A')
                player[0], player[ace_index] = player[ace_index], player[0]
                
            card_values = [11, deck[player[1]][1]]
        
        else:
            card_values = [deck[card][1] for card in player]
            
    if len(player) > 3:
        if 'A' in player:
            
    
    return sum(card_values)      

IndentationError: expected an indented block (<ipython-input-118-95bfd5931949>, line 21)

In [4]:
# These next modules were part of a failed attempt; above CF works. Left it here for notes purposes; and will be cleared up
# in a review. 

# Two stages to this module.
# 1. Take the player hand as input and compute and save the score, to be used 
# for evaluation of outcome.

# 2. Feeding player hand into decision and action module - should this be
# located within the module or outside the module?

# This module has to be able to accomodate player hands of different lengths.

In [36]:
# For two card hands, look up values of each card in the dictionary and
# compute the sum. 

# This is used when earlier decision module outputs decision = {S}

# Q: For two card hands with an ace - what cases need to be considered?
# A: Yellow decision = {S} squares.

# AA - comes under pair, any in any case, a two card [A, A] which then
# proceeds to dealer turn would not occur under BS. Every decision
# associated with AA would lead to further splits, dealt with later,
# or a hit, making it a three card hand.

# A2 - A6 - all decisions are {H, D} so these do not need to be considered.
# *A10, AJ, AQ, AK not specified as that would mean player blackjack - needs to be resolved.

# A7 has 3 instances of decision = {S} which would mean a two card A7 hand
# A8 - A9 has decision = {S} on every instance which would mean two card A8, A9 hands
# For all two card instances where the BS specifies decision = {S}, it can
# safely assumed that the ace takes on the value 11. 

# For two card hands that could have yielded a split - what cases?
# Under BS, only way in which two card hands containing a pair can pass through
# this module is (9,9) and (10,10) for which decision = {S}.
# So this case can be bundled with a standard dictionary look up and sum()

# *Later consideration - should I design this with a view to modularity?
# That is should the new modules at each update be able to accomodate
# the behaviour of previous modules?
# Ideally yes, but I think you need to focus on getting a working module for
# the more complex case done first before you start worrying about design
# considerations where complex collapses into simple. #

# Efficiency considerations later; get working code set up and optimisation later

In [34]:
# Two card check_bust_score check score module completed.

# ??? - You got a little confused here. It is not possible to go bust with a 2 card hand....!!!
# Evaluate what went wrong with your thinking here during debrief write-up.
# Actually this is fine, it's a module to compute the score. 
# Thinking was that computing score will be be necessary to determine
# if a player is bust. And also that it would be needed anyway.

# Clarifying the broader CF really helped here. 

def check_bust_score_module(player):
    if 'A' in player:
        if player[0] != 'A':
            ace_index = player.index('A')
            player[0], player[ace_index] = player[ace_index], player[0]
                
        card_values = [11, deck[player[1]][1]]
        
    else:
        card_values = [deck[card][1] for card in player]
        
    return sum(card_values)

In [35]:
check_bust_score_module(['A', 'A'])

12

In [37]:
# For three card hands, deal with simplest case where there are no aces
# Look up the values in dictionary and add them up.

# This module will be used when the earlier decision module outputs d = {H, D}

# Consulting BS table.

# Case bashing/rules - 3 card case

# Clarity on the edge case [A, A, A] - possible in one case.

# If [X, Y, Z] does not contain ace, then it's just a sum.

# For three card hands containing an ace:
# [A, X, Y] - ace takes value 1 if (X + Y) >= 11. 
# [X, A, Y] and [X, Y, A] - same, but put A at front of list so sum(X+Y) can
# be computed. 
# So for these cases, it becomes a hard total, as the ace can only take the value 1
# 

# [A, X, Y] - ace can take value 1 or 11 if (X + Y) <= 10
# Score can be computed, but can take two values. 
# In this case, the hand is definitely not bust.
# Is it meaningful to compute the score? And how should we treat it? BS table


In [ ]:
# Extend this to hands of greater than length 2

def check_bust_score_module(player):
    if 'A' in player:
        if player[0] != 'A':
            ace_index = player.index('A')
            player[0], player[ace_index] = player[ace_index], player[0]
            
        cardvalues_no_ace = [deck[card][1] for card in player[1::]]
        
        if sum(cardvalues_no_ace) <= 10:
            ? # if the next 
        else:
            card_values = [1, sum(cardvalues_no_ace)]
    else:
        card_values = [deck[card][1] for card in player]

In [ ]:
# Q: Might it be a better idea to separate the check bust and compute player score modules? 

# Q: Examine relationship between score computation and checking if bust. You have been *assuming that they naturally go together. 

# Problem - it is not actually possible to say what the score is for the above case where there is a question mark. 

# Had another think about the functionality of the check_bust_score_module - it needed clarifying, and CF needed to be scrutinised.
# Context PD = {H}. If an earlier decision was to hit and a card was drawn, the CF needs to CHECK if the player is bust.

# Otherwise, if we were to pass the player hand back to the decision module after the hand was bust, it wouldn't make any sense.
# E.g. if player = [A, 3, 9] and dealer = [J], which is clearly bust, the player no longer can make any decisions and crucially,
# the CF takes over. 

# If player is not bust, we need to feed his hand back into another decision-action sequence. This continues *indefinitely*,
# with termination/escape from the loop only possible if either the decision becomes a stand or double {S,D}, OR the player busts.

# Context PD = {S, D}. Evaluation of whether or not the player is bust is *irrelevant* for CF/player decision purposes. That
# is because there is NO feedback into another decision-action sequence and the CF takes over.
# However, score computation and determining whether the player is bust will be needed for evaluating hand outcome. 

